In [55]:
from pyomo.environ import *
import numpy as np
import random
import math
import pandas as pd
# General information
weather_data = pd.read_csv (r'C:\Users\aliam\Documents\University\PhD\Courses\Computational Optimization\Project\Data\NYC_Hourly_Normal_Temp.csv')
#print(weather_data)
NG = 2 # Number of generators
NW = 1 # Number of wind farms
NL = 2 # Number of loads
PG_down = np.zeros(NG)
PG_up = np.array([163,85]) # BO: please correct/complete it
#CG = np.array    Bo
#CL = np.array   Bo
#CW = np.array Bo
#B = Bo
#F = Bo
Tamb = (weather_data.iloc[:,8] - 32)/1.8
Tamb_10PCTL = (weather_data.iloc[:,6] - 32)/1.8
Tamb_90PCTL = (weather_data.iloc[:,7] - 32)/1.8
std_Tamb_10PCTL = (Tamb - Tamb_10PCTL)/1.282 
std_Tamb_90PCTL = (Tamb_90PCTL - Tamb)/1.282 
std_Tamb = (std_Tamb_10PCTL + std_Tamb_90PCTL)/2
#PW = Bo 
# Parameters related to the demand side
num_TCL = 1000
Tset_set = [21,21.5,22,22.5,23]
Tset = []
for i in range(num_TCL):
    s = random.sample(Tset_set,1)
    s = s[0]
    Tset.append(s)
delta = 1
R_nom = 2
R_low = 0.8*R_nom
R_high = 1.2*R_nom
C_nom = 2
C_low = 0.8*C_nom
C_high = 1.2*C_nom
P_el_nom = 5.6
P_el_low = 0.8*P_el_nom
P_el_high = 1.2*P_el_nom
COP_nom = 2.5
COP_low = 0.8*COP_nom
COP_high = 1.2*COP_nom
R = np.random.uniform(R_low,R_high,num_TCL)
C = np.random.uniform(C_low,C_high,num_TCL)
P_el = np.random.uniform(P_el_low,P_el_high,num_TCL)
COP = np.random.uniform(COP_low,COP_high,num_TCL)
# Sampling
eps = 0.1
beta = 0.1
num_decision_variables = 5*NG + 5*NL
num_samples = math.ceil((2/eps)*(np.log(1/beta) + num_decision_variables))
i = 0
mu_Tamb = Tamb[i]
sigma_Tamb = std_Tamb[i] 
Tamb_sample = np.random.normal(mu_Tamb,sigma_Tamb,num_samples)
for i in range(num_samples):
    identified_VS = VS(0.25,Tamb_sample[i],num_TCL,Tset,delta,R,C,P_el,COP)
    #print(identified_VS[0])
#print(num_samples)
#PW_sample = random.sample(PW,num_samples)

In [33]:
def VS(dt,Tamb,num_TCL,Tset,delta,R,C,P_el,COP):
    Tdown = np.zeros(num_TCL)
    Tup = np.zeros(num_TCL)
    T_ON = np.zeros(num_TCL)
    T_OFF = np.zeros(num_TCL)
    D = np.zeros(num_TCL)
    B = 0
    Pdown = 0
    Pup = 0
    Sdown = 0
    Sup = 0
    for i in range(0,num_TCL):
        Tdown[i] = Tset[i] - delta/2
        Tup[i] = Tset[i] + delta/2
        T_ON[i] = C[i]*R[i]*np.log((Tdown[i] - Tamb - R[i]*P_el[i]*COP[i])/(Tup[i] - Tamb - R[i]*P_el[i]*COP[i]));
        T_OFF[i] = C[i]*R[i]*np.log((Tup[i] - Tamb)/(Tdown[i] - Tamb));
        D[i] = T_ON[i]/(T_ON[i] + T_OFF[i])
        if ( Tamb < Tdown[i] and Tamb + R[i]*P_el[i]*COP[i] > Tup[i] ):
            B = B + P_el[i]*D[i]
            Pup = Pup + P_el[i]
            Sup = Sup + dt*P_el[i]*(1 - D[i])
    return [B,Pdown,Pup,Sdown,Sup]

In [56]:
# Optimization: Problem formulation
def optimizer(NG,NL,Tamb_sample,num_TCL,Tset,delta,R,C,P_el,COP,S0):
    model = ConcreteModel()
    # Defining the variables
    model.PG = Var(range(NG),within = NonNegativeReals)
    model.PC = Var(range(NL),within = NonNegativeReals)
    model.RG_down = Var(range(NG),within = NonNegativeReals)
    model.RG_up = Var(range(NG),within = NonNegativeReals)
    model.RG = Var(range(NG),within = NonNegativeReals)
    model.RL_down = Var(range(NL),within = NonNegativeReals)
    model.RL_up = Var(range(NL),within = NonNegativeReals)
    model.RL = Var(range(NL),within = NonNegativeReals)
    model.dG_down = Var(range(NG),within = NonNegativeReals)
    model.dG_up = Var(range(NG),within = NonNegativeReals)
    model.dL_down = Var(range(NL),within = NonNegativeReals)
    model.dL_up = Var(range(NL),within = NonNegativeReals)
    # Defining the constraints
    model.c = ConstraintList()
    for i in range(NG):
        model.c.add(model.PG[i] + model.RG[i] >= PG_down[i])
        model.c.add(model.PG[i] + model.RG[i] <= PG_up[i])
        model.c.add(model.RG[i] >= -model.RG_down[i])
        model.c.add(model.RG[i] <= model.RG_up[i])
        for i in range(NL):
            for j in range(num_samples):
                identified_VS = VS(0.25,Tamb_sample[j],num_TCL,Tset,delta,R,C,P_el,COP)
                model.c.add(model.PC[i] + model.RL[i] >= identified_VS[1])
                model.c.add(model.PC[i] + model.RL[i] <= identified_VS[2])
                model.c.add(model.RL[i] >= -model.RL_down[i])
                model.c.add(model.RL[i] <= model.RL_up[i])
                model.c.add(S0 + 0.25*(model.PC[i] + model.RL[i] - identified_VS[0]) >= identified_VS[3])
                model.c.add(S0 + 0.25*(model.PC[i] + model.RL[i] - identified_VS[0]) <= identified_VS[4])
                model.c.add(S0 + 0.75*(model.PC[i] - identified_VS[0]) + 0.25*(model.PC[i] + model.RL[i] - identified_VS[0]) >= identified_VS[3])
                model.c.add(S0 + 0.75*(model.PC[i] - identified_VS[0]) + 0.25*(model.PC[i] + model.RL[i] - identified_VS[0]) <= identified_VS[4])
                # Objective function
                model.obj = Objective(rule = model.alpha,sense = minimize)
                model.pprint()
                solver = SolverFactory('glpk')
                solver.solve(model)
                return model  